Kevin Chen

In [57]:
# Import The Appropriate Libraries
import pandas as pd
import math

# Data Table Creation
d = {
    'Tolerance/Eps':[],
    'AdaptiveSimpson':[],
    'Exact Value':[],
    'Difference':[]
}

# The exact result
def exactvalue(x):
    exact = (1/2)*(x+((1/2)*math.sin(2*x)))
    return exact

In [58]:
# Textbook Adaptive Quadrature with the Composite Simpson's Rule described in Section 4.6 of the textbook
# Pseudocode
def f(x):
    result = math.cos(x)**2
    return result

def adaptiveQuadrature(a, b, tol, n):
    toli = []
    ai = []
    hi = []
    fai = []
    fci = []
    fbi = []
    si = []
    li = []
    
    app = 0
    i = 0
    toli.append(10* tol)
    ai.append(a)
    hi.append((b-a)/2)
    fai.append(f(a))
    fci.append(f(a+hi[i]))
    fbi.append(f(b))
    si.append(hi[i]*(fai[i]+(4*fci[i])+fbi[i])/3) # Approximation from Simpson's method for entire interval
    li.append(1)
    
    while i > -1:
        fd = f(ai[i]+(hi[i]/2))
        fe = f(ai[i]+(3*hi[i]/2))
        s1 = hi[i]*(fai[i]+(4*fd)+fci[i])/6 # Approximation from Simpson's method for halves of subintervals
        s2 = hi[i]*(fci[i]+(4*fe)+fbi[i])/6
        v1 = ai[i] # Save data at this level
        v2 = fai[i]
        v3 = fci[i]
        v4 = fbi[i]
        v5 = hi[i]
        v6 = toli[i]
        v7 = si[i]
        v8 = li[i]
        i = i - 1 # Delete the level
        if (abs(s1 + s2 - v7) < v6):
            app = app + (s1 + s2)
        else:
            if (v8 >= n):
                print('Level Exceeded') # Procedure fails
                break
            else: # Add one level
                i = i + 1 # Data for Right-Half Subinterval
                ai[i] = v1 + v5
                fai[i] = v3
                fci[i] = fe
                fbi[i] = v4
                hi[i] = v5/2
                toli[i] = v6/2
                si[i] = s2
                li[i] = v8 + 1
                i = i + 1 # Data for Left-Half Subinterval
                ai.append(v1)
                fai.append(v2)
                fci.append(fd)
                fbi.append(v3)
                hi.append(hi[i-1])
                toli.append(toli[i-1])
                si.append(s1)
                li.append(li[i-1])
    return app

In [59]:
# https://rosettacode.org/wiki/Numerical_integration/Adaptive_Simpson%27s_method#Python
# Code obtained from site above

import collections
triple = collections.namedtuple('triple', 'm fm simp')
 
def _quad_simpsons_mem(f: callable, a: float , fa: float, b: float, fb: float)->tuple:
    # Evaluates Simpson's Rule, also returning m and f(m) to reuse.
    m = a + (b - a) / 2
    fm = f(m)**2
    simp = abs(b - a) / 6 * (fa + 4*fm + fb)
    return triple(m, fm, simp,)
 
def _quad_asr(f: callable, a: float, fa: float, b: float, fb: float, eps: float, whole: float, m: float, fm: float)->float:
    # Efficient recursive implementation of adaptive Simpson's rule.
    # Function values at the start, middle, end of the intervals are retained.
    lt = _quad_simpsons_mem(f, a, fa, m, fm)
    rt = _quad_simpsons_mem(f, m, fm, b, fb)
    delta = lt.simp + rt.simp - whole
    return (lt.simp + rt.simp + delta/15
        if (abs(delta) <= eps * 15) else
            _quad_asr(f, a, fa, m, fm, eps/2, lt.simp, lt.m, lt.fm) +
            _quad_asr(f, m, fm, b, fb, eps/2, rt.simp, rt.m, rt.fm)
    )
 
def quad_asr(f: callable, a: float, b: float, eps: float)->float:
    # Integrate f from a to b using ASR with max error of eps.
    fa = f(a)**2
    fb = f(b)**2
    t = _quad_simpsons_mem(f, a, fa, b, fb)
    return _quad_asr(f, a, fa, b, fb, eps, t.simp, t.m, t.fm)
 
def Adaptive_Simpson(a,b,tol):
    result = quad_asr(math.cos, a, b, tol);
    return result

In [60]:
a = 1
b = 3
n = 10

for k in range(1,10):
    tol = pow(10,-k)
    d['Tolerance/Eps'].append(tol)
    
    AdaptiveSimpson = Adaptive_Simpson(a,b,tol)
    d['AdaptiveSimpson'].append(AdaptiveSimpson)

    exact = exactvalue(b) - exactvalue(a)
    d['Exact Value'].append(exact)
    d['Difference'].append(abs(exact-AdaptiveSimpson))

In [61]:
# Creates a Table

display(pd.DataFrame(d, columns = ['Tolerance/Eps','AdaptiveSimpson','Exact Value','Difference']))

,Tolerance/Eps,AdaptiveSimpson,Exact Value,Difference
0,1.000000e-01,0.704021,0.702822,1.199017e-03
1,1.000000e-02,0.704021,0.702822,1.199017e-03
2,1.000000e-03,0.702833,0.702822,1.127142e-05
3,1.000000e-04,0.702817,0.702822,4.304127e-06
4,1.000000e-05,0.702822,0.702822,2.899710e-08
5,1.000000e-06,0.702822,0.702822,5.575806e-10
6,1.000000e-07,0.702822,0.702822,2.898792e-13
7,1.000000e-08,0.702822,0.702822,1.262990e-12
8,1.000000e-09,0.702822,0.702822,1.365574e-14


We can see from the table above that as the value of 'Tolerance/Eps' decreases, the value of the 'AdaptiveSimpson' becomes closer to the 'Exact Value'. This could also be seen by comparing the 'Tolerance/Eps' with the 'Difference'. We know this is true because from section 4.6 of the textbook on adaptive quadrature with composite simpson's rule, there would be less room for errors/tolerance, making the adaptive quadrature method closer to the exact value.